In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
'''# Rename pictures so we can trace them better after classification
dir_list = ['data/test/shoe', 'data/test/no_shoe']

for dir in dir_list:
    i = 1
    try:
        for filename in os.listdir(dir): 
            if filename == 'Thumbs.db':
                continue
            dst = 'test_pictures' + '_' + str(i) + ".jpg"
            src = dir + '/' + filename 
            dst = dir + '/' + dst 

            # rename() function will 
            # rename all the files 
            os.rename(src, dst) 
            i += 1
    except Exception as e:
        print(e)
    finally:
        i += 1

        print("formatted " + str(dir))'''

'# Rename pictures so we can trace them better after classification\ndir_list = [\'data/test/shoe\', \'data/test/no_shoe\']\n\nfor dir in dir_list:\n    i = 1\n    try:\n        for filename in os.listdir(dir): \n            if filename == \'Thumbs.db\':\n                continue\n            dst = \'test_pictures\' + \'_\' + str(i) + ".jpg"\n            src = dir + \'/\' + filename \n            dst = dir + \'/\' + dst \n\n            # rename() function will \n            # rename all the files \n            os.rename(src, dst) \n            i += 1\n    except Exception as e:\n        print(e)\n    finally:\n        i += 1\n\n        print("formatted " + str(dir))'

In [3]:
# Load model
model = load_model('models/first_model/model.h5')
model.load_weights('models/second_model_with_regularization/weights-improvement-50-0.96.hdf5')

In [4]:
# Create generator to pull pictures from directory
datagen = ImageDataGenerator(rescale=1./255)
generator = datagen.flow_from_directory(
        'data/test',
        target_size=(300, 300),
        batch_size=10,
        shuffle=False)

probabilities= model.predict_generator(generator, 50,  verbose=1)

Found 174 images belonging to 2 classes.
 1/50 [..............................] - ETA: 1:50

C:\Users\Lucas\Anaconda3\envs\TFEnv\lib\site-packages\PIL\Image.py:966: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


50/50 [==============================] - 70s 1s/step


In [5]:
# Zip predictions to filenames
mapped_preds = np.array(list(zip(probabilities.flatten(),np.asarray(generator.filenames))))
mapped_preds_list = mapped_preds.tolist()
# mapped_preds_list

In [6]:
# Pack predictions, correct label, file name and predicted probability in a list
def map_correct(prediction_touple):
    if float(prediction_touple[0]) < 0.5:
        prediction = 0
    else:
        prediction = 1
    
    if prediction_touple[1][0] == 's':
        correct_pred = 1
    else:
        correct_pred = 0
    
    return list([prediction, correct_pred, prediction_touple[1], prediction_touple[0]])

In [7]:
# Reformat to dataframe
list_for_df = list(map(map_correct, mapped_preds_list))
evaluation_df = pd.DataFrame(list_for_df, columns = ['prediction' , 'correct label', 'file name', 'predicted probability']) 
# evaluation_df

In [8]:
# Classification report
print(classification_report(evaluation_df['correct label'], evaluation_df['prediction']))

              precision    recall  f1-score   support

           0       0.71      0.59      0.65        37
           1       0.90      0.93      0.91       137

   micro avg       0.86      0.86      0.86       174
   macro avg       0.80      0.76      0.78       174
weighted avg       0.86      0.86      0.86       174



In [9]:
# Print complete df
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(evaluation_df)

     prediction  correct label                     file name  \
0             1              0   no_shoe\test_pictures_1.jpg   
1             0              0  no_shoe\test_pictures_10.jpg   
2             0              0  no_shoe\test_pictures_11.jpg   
3             1              0  no_shoe\test_pictures_12.jpg   
4             0              0  no_shoe\test_pictures_13.jpg   
5             0              0  no_shoe\test_pictures_14.jpg   
6             0              0  no_shoe\test_pictures_15.jpg   
7             0              0  no_shoe\test_pictures_16.jpg   
8             0              0  no_shoe\test_pictures_17.jpg   
9             0              0  no_shoe\test_pictures_18.jpg   
10            0              0  no_shoe\test_pictures_19.jpg   
11            1              0   no_shoe\test_pictures_2.jpg   
12            1              0  no_shoe\test_pictures_20.jpg   
13            1              0  no_shoe\test_pictures_21.jpg   
14            1              0  no_shoe\

In [10]:
# Print confusion matrix
confusion_matrix(evaluation_df['correct label'], evaluation_df['prediction'])

array([[ 22,  15],
       [  9, 128]], dtype=int64)